In [1]:
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', None)

In [2]:
path = Path('../data/runwayData/') # Cargamos la utilizacion de las pistas para cada intervalo de 30 minutos
files = list(path.glob('*.csv'))
print(f"{len(files)} Archivos cargados =======")

36761 Archivos cargados =======


In [3]:
def runwaysInUse(row):
    return len([x for x in row if x > 0])

In [4]:
# Cargamos los datos de ocupacion de todos los intervalos de 30 minutos y mapeamos la ocupacion.

config = []
read = []
print("Comienzo carga de dataFrames")
for idx, f in enumerate(files):
    df = pd.read_csv(f , sep=';')
    op = []
    count = runwaysInUse(df['NumberOperations'].values)
    configuration = f'{count}|'
    for index, row in df.iterrows():
        if row['NumberOperations'] > 0:
            configuration += f"{row['Type'][0]}.{row['Runway']}-"
            op.append(f"{row['NumberOperations']}#{row['Type'][0]}.{row['Runway']}")

    configuration = configuration[:-1]

    config.append(configuration)
    read.append((f.stem , configuration , op)) # (Hora, Configuracion, Operaciones)
    
    if idx % 1500 == 0:
        print(f"(cargaDataFrame) Se han cargado {idx + 1} de {len(files)} dataFrames")
    
print("Fin carga de dataFrames")

Comienzo carga de dataFrames
(cargaDataFrame) Se han cargado 1 de 36761 dataFrames
(cargaDataFrame) Se han cargado 1501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 3001 de 36761 dataFrames
(cargaDataFrame) Se han cargado 4501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 6001 de 36761 dataFrames
(cargaDataFrame) Se han cargado 7501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 9001 de 36761 dataFrames
(cargaDataFrame) Se han cargado 10501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 12001 de 36761 dataFrames
(cargaDataFrame) Se han cargado 13501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 15001 de 36761 dataFrames
(cargaDataFrame) Se han cargado 16501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 18001 de 36761 dataFrames
(cargaDataFrame) Se han cargado 19501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 21001 de 36761 dataFrames
(cargaDataFrame) Se han cargado 22501 de 36761 dataFrames
(cargaDataFrame) Se han cargado 24001 de 36761 dataFr

In [5]:
def maped_madrid_runways(runway_list, sensibility = 0.6):
    c = {}
    i = 0
    nort_conf = "A.32R-A.32L-D.36L-D.36R"
    sur_conf = "A.18R-A.18L-D.14L-D.14R"

    with open (f"../data/resultados/sensibilidad/resume{str(sensibility).replace('.', ';')}.csv", 'w', encoding='utf-8') as file:
        file.write("Day;Configuracion;NOperations;SOperations;TotalOperations\n")

    for day, configuration, usage in runway_list:
        hour = int(day.split(' ')[1].replace(';', ':').split(':')[0])
    
        total_operations, nOperation, sOperation = 0, 0, 0
        for runways in usage: # Calculamos las operaciones que se realizan en cada una de las pistas
            operation, runway = runways.split('#')
            if runway in nort_conf:
                nOperation += int(operation)
            elif runway in sur_conf:
                sOperation += int(operation)
            total_operations = nOperation + sOperation
        
        # Asignamos a cada hora una configuracion en funcion de las pistas que se utilicen
        
        if nOperation > 0 and sOperation == 0: 
             with open (f"../data/resultados/sensibilidad/resume{str(sensibility).replace('.', ';')}.csv", 'a', encoding='utf-8') as file:
                file.write(f"{day.replace('Operation', '').replace(';', ':')};NORTE;{nOperation};{sOperation};{total_operations}\n")
        elif sOperation > 0 and nOperation == 0:
             with open (f"../data/resultados/sensibilidad/resume{str(sensibility).replace('.', ';')}.csv", 'a', encoding='utf-8') as file:
                file.write(f"{day.replace('Operation', '').replace(';', ':')};SUR;{nOperation};{sOperation};{total_operations}\n")
        elif nOperation > 0 and sOperation > 0:
            if nOperation / total_operations > sensibility:
                with open (f"../data/resultados/sensibilidad/resume{str(sensibility).replace('.', ';')}.csv", 'a', encoding='utf-8') as file:
                    file.write(f"{day.replace('Operation', '').replace(';', ':')};NORTE;{nOperation};{sOperation};{total_operations}\n")
            elif sOperation / total_operations > sensibility:
                with open (f"../data/resultados/sensibilidad/resume{str(sensibility).replace('.', ';')}.csv", 'a', encoding='utf-8') as file:
                    file.write(f"{day.replace('Operation', '').replace(';', ':')};SUR;{nOperation};{sOperation};{total_operations}\n")
            else:
                with open (f"../data/resultados/sensibilidad/resume{str(sensibility).replace('.', ';')}.csv", 'a', encoding='utf-8') as file:
                    file.write(f"{day.replace('Operation', '').replace(';', ':')};AMBAS;{nOperation};{sOperation};{total_operations}\n")
        elif nOperation == 0 and sOperation == 0:
            pass
        else:
            with open (f"../data/resultados/sensibilidad/resume{str(sensibility).replace('.', ';')}.csv", 'a', encoding='utf-8') as file:
                file.write(f"{day.replace('Operation', '').replace(';', ':')};ERROR;{nOperation};{sOperation};{total_operations}\n")

In [6]:
sensibility = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

for idx, s in enumerate(sensibility): # Realizamos el mapeo de la configuracion para distintos valores de sensibilidad
    try:
        maped_madrid_runways(read , sensibility=s)
    except Exception as e:
        print(f"ERROR. SE HA PRODUCIDO UN ERROR CON LA SENSIBILIDAD {s}\nCodigo: {e}")
        print(f"Fin del analisis sensibilidad numero {idx + 1}!!!")
        continue
print("Fin de todos los analisis sensibilidad!!!!!!")

Fin de todos los analisis sensibilidad!!!!!!


### Ordenamos los registros del .csv por día y añadimos el mapeo de las configuraciones

In [7]:
path2 = Path('../data/resultados/sensibilidad')
files2 = list(path2.glob('*.csv'))
print(f"{len(files2)} Archivos cargados =======")

10 Archivos cargados =======


In [8]:
mapping = {"NORTE":1, "SUR":1, "AMBAS":2}
for f in files2:  # Ordenamos el dataset por fecha
    df = pd.read_csv(f, sep = ";")
    df = df.sort_values(by=['Day'])
    colum = []
    for index, row in df.iterrows():
        colum.append(mapping[row["Configuracion"]])
    
    df["ConfigNorm"] = colum
    df.to_csv(f"{path2}/{f.stem}.csv", sep=';', index=False)